In [19]:
# あとで顔検出など使いたくなる可能性あるので、最初からOpenCVでやる
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.optimizers import Adam

import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image


# ラベル付け(0春 1夏 2冬)
# TODO: ? フォルダ名に数字入れておくとかのほうがいいかもしれない？
def get_label(value):
    if value == "Ariel":
        return 0
    elif value == "Other":
        return 1
    
# ラベル更新したら一緒に更新する
def max_label():
    return 2
    

# 教師データ
train_images = []
train_labels = []

# Image直下のフォルダを網羅（フォルダ名からファイルにラベル付けする）
train_path = "../image/train"
for dir in os.listdir(train_path):
    # Windows環境だから無いけど念の為
    if dir == ".DS_Store":
        continue
        
    dir_path = train_path + "/" + dir
    label = get_label(dir)

    for file in os.listdir(dir_path):
        # Windows環境だから無いけど念の為
        # TODO: というか画像以外は無視するようにしたい
        if dir != ".DS_Store":
            file_path = dir_path + "/" + file
            
            # 正解ラベルの追加
            train_labels.append(label)
            
            # 画像を1080x1080の正方形にリサイズ（機種によって縦長横長が混在しているので正方形）
            # 1要素が[R,G,B]3要素を含む配列の25x25の２次元配列として読み込む
            # [R,G,B]はそれぞれが0-255の配列。
            image = np.array(Image.open(file_path).resize(size=(1024, 1024)))
            # 配列を変換し、[[Redの配列],[Greenの配列],[Blueの配列]] のような形にする。
            image = image.transpose(2, 0, 1)
            # さらにフラットな1次元配列に変換。最初の1/3はRed、次がGreenの、最後がBlueの要素がフラットに並ぶ。
            image = image.reshape(image.shape[0] * image.shape[1] * image.shape[2], 1)
            # 出来上がった配列をimage_listに追加。
            train_images.append(image / 255.)
            # ほぼ写経 https://zenn.dev/hiroe_orz17/articles/53af65f491122e
            
# kerasに渡すためにnumpy配列に変換        
train_images = np.array(train_images)

# ラベルの配列を1と0からなるラベル配列に変更
# 0 -> [1,0], 1 -> [0,1] という感じ。
Y = to_categorical(train_labels)

# ラベル配列の中身を見たいときはコレ
# np.set_printoptions(threshold=np.inf)
# print(Y)

# モデルの構築
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(3145728,1)),
    keras.layers.Dense(200),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(200),
    keras.layers.Activation("relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(max_label()),
    keras.layers.Activation("softmax")
])
print(model.summary())

# オプティマイザにAdamを使用
opt = Adam(learning_rate=0.001)

# モデルをコンパイル
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
# 学習を実行。10%はテストに使用。
model.fit(train_images, Y, epochs=30, batch_size=100, validation_split=0.1)



[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 

In [20]:
# テスト用ディレクトリの画像でチェック。正解率を表示する。
total = 0.
ok_count = 0.

test_path = "../image/test"
for dir in os.listdir(test_path):
    if dir == ".DS_Store":
        continue

    dir_path = test_path + "/" + dir
    label = get_label(dir)

    for file in os.listdir(dir_path):
        if file != ".DS_Store":
            print("--------------------------------------------------------")
            print(file)
            file_path = dir_path + "/" + file
            image = np.array(Image.open(file_path).resize(size=(1024, 1024)))
            image = image.transpose(2, 0, 1)
            image = image.reshape(image.shape[0] * image.shape[1] * image.shape[2], 1)
#             image = image / 255.
#             print(image.shape)
            result = model.predict(np.array([image / 255.]))
            print(result)
            print("label:", label, "result:", result[0])

#             total += 1.

#             if label == result[0]:
#                 ok_count += 1.

# print("seikai: ", ok_count / total * 100, "%")

--------------------------------------------------------
20200206-134540_MobileL7_100000000_0.00_0.00_C249-SET-000-XXX_225.bmp
[[0.44395047 0.5560495 ]]
label: 0 result: [0.44395047 0.5560495 ]
--------------------------------------------------------
20200518-202340_MobileL4_100000000_0.00_0.00_C249-SET-000-XXX_205.bmp
[[0.44395047 0.5560495 ]]
label: 0 result: [0.44395047 0.5560495 ]
--------------------------------------------------------
20200519-095934_MobileL4_100000000_0.00_0.00_C249-SET-000-XXX_185.bmp
[[0.44395047 0.5560495 ]]
label: 0 result: [0.44395047 0.5560495 ]
--------------------------------------------------------
20200520-154352_MobileL2_100000000_0.00_0.00_C249-SET-000-XXX_220.bmp
[[0.44395047 0.5560495 ]]
label: 0 result: [0.44395047 0.5560495 ]
--------------------------------------------------------
20210413-131911_MobileL3_100000000_0.00_0.00_HAPPY-SET-000-XXX_2番目A_1番目B_.bmp
[[0.44395047 0.5560495 ]]
label: 1 result: [0.44395047 0.5560495 ]
----------------------